In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [144]:
from os import getcwd, path, pardir
import pickle
from math import ceil

In [145]:
run_experiments = False

Experiment AES

In [146]:
if run_experiments:
    from Security.AES.experiment_aes import experiment_aes
    experiment_aes(path.join(getcwd(), "Security", "AES"))

encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_encrypt_charge.p"), "rb"))
decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES", "aes_decrypt_charge.p"), "rb"))

Experiment AES Flash

In [147]:
if run_experiments:
    from Security.AES_Flash.experiment_aes_flash import experiment_aes_flash
    experiment_aes_flash(path.join(getcwd(), "Security", "AES_Flash"))

aes_encrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_encrypt_charge.p"), "rb"))
aes_decrypt_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_decrypt_charge.p"), "rb"))
flash_write_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_write_charge.p"), "rb"))
flash_read_charge = pickle.load(open(path.join(getcwd(), "Security", "AES_Flash", "aes_flash_read_charge.p"), "rb"))

Experiment TrustZone

In [148]:
if run_experiments:
    from Security.TrustZone.experiment_trustzone import experiment_trustzone
    experiment_trustzone(path.join(getcwd(), "Security", "TrustZone"))

nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_store_charge.p"), "rb"))
nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone", "trustzone_load_charge.p"), "rb"))

Experiment TrustZone Flash

In [149]:
if run_experiments:
    from Security.TrustZone_Flash.experiment_trustzone_flash import experiment_trustzone_flash
    experiment_trustzone_flash(path.join(getcwd(), "Security", "TrustZone_Flash"))

flash_nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
flash_nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))

Make lookup function

In [150]:
AES_BLOCK_SIZE = 16
def get_security_charge(security_type, number_of_bytes):
    if security_type is "AES":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return encrypt_charge[index] + decrypt_charge[index]
    elif security_type is "AES_Flash":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return aes_encrypt_charge[index] + flash_write_charge[index] + flash_read_charge[index] + aes_decrypt_charge[index]
    elif security_type is "TrustZone":
        return nsc_store_charge[number_of_bytes] + nsc_store_charge[number_of_bytes]
    elif security_type is "TrustZone_Flash":
        return flash_nsc_store_charge[number_of_bytes] + flash_nsc_load_charge[number_of_bytes]
    else:
        return 0 # No security, so 0 C

In [165]:
security_types = ["None", "AES", "AES_Flash", "TrustZone", "TrustZone_Flash"]
# security_types = ["None", "AES_Flash", "TruseZone_Flash"]
application_names = ["Kalman", "AR", "Dijkstra", "Sorting", "Bit Count", "fft"]

In [166]:
app_charge_scale = 1e-4
application_charges = np.array([1, 2, 3, 4, 5 ,6]) * app_charge_scale # TODO
# app_number_of_bytes_scale = 1000
application_number_of_bytes = np.array([234, 32, 876, 12, 90, 95]) # TODO

In [167]:
base_charge_per_app = dict(zip(application_names, application_charges))
number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

In [168]:
def get_total_charge(application_name, security_type):
    print(application_name, security_type, f"base: {base_charge_per_app[application_name]}, sec: {get_security_charge(security_type, number_of_bytes_per_app[application_name])}")
    return base_charge_per_app[application_name] + get_security_charge(security_type, number_of_bytes_per_app[application_name])

# Bar chart

In [169]:
import numpy as np
import matplotlib.pyplot as plt

In [170]:
# create plot
fig, ax = plt.subplots(figsize=(10,9))
index = np.arange(len(application_names))
bar_width = 0.18
opacity = 0.8

# ax.grid()

colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

rects = []
for i,security_type in enumerate(security_types):
    rects += ax.bar(index + i*bar_width, [get_total_charge(application_name, security_type) for application_name in application_names], bar_width, label=security_type, color=colors[i], alpha=opacity)
 
ax.set_xlabel('Application')
ax.set_ylabel('Charge [C]')
ax.set_title('Charge per Application')
fig.canvas.set_window_title('Charge per Application')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(application_names)
ax.legend()
 
fig.tight_layout()
fig.show()

Kalman None base: 0.0001, sec: 0
AR None base: 0.0002, sec: 0
Dijkstra None base: 0.00030000000000000003, sec: 0
Sorting None base: 0.0004, sec: 0
Bit Count None base: 0.0005, sec: 0
fft None base: 0.0006000000000000001, sec: 0
Kalman AES base: 0.0001, sec: 1.665989470970225e-05
AR AES base: 0.0002, sec: 3.1193849111543757e-06
Dijkstra AES base: 0.00030000000000000003, sec: 5.856134925316661e-05
Sorting AES base: 0.0004, sec: 2.052742179666275e-06
Bit Count AES base: 0.0005, sec: 7.312193583620544e-06
fft AES base: 0.0006000000000000001, sec: 7.312193583620544e-06
Kalman AES_Flash base: 0.0001, sec: 2.4004835237013053e-05
AR AES_Flash base: 0.0002, sec: 7.896681360580167e-06
Dijkstra AES_Flash base: 0.00030000000000000003, sec: 8.410964901737793e-05
Sorting AES_Flash base: 0.0004, sec: 6.771078408967512e-06
Bit Count AES_Flash base: 0.0005, sec: 1.2851533253425208e-05
fft AES_Flash base: 0.0006000000000000001, sec: 1.2851533253425208e-05
Kalman TrustZone base: 0.0001, sec: 4.5780308917

# Number of Checkpoints

In [157]:
security_types = ["None", "AES_Flash", "TruseZone_Flash"]

In [158]:
cap_scale_per_app = 1e1
points_per_unit = 100
capacitor_sizes = np.arange(app_charge_scale*cap_scale_per_app,app_charge_scale*cap_scale_per_app*2, app_charge_scale*cap_scale_per_app/points_per_unit)
print(min(capacitor_sizes),max(capacitor_sizes))
print(base_charge_per_app["Kalman"])
print(get_security_charge("None", number_of_bytes_per_app["Kalman"]))
print(get_security_charge("AES_Flash", number_of_bytes_per_app["Kalman"]))

0.001 0.0019900000000000026
0.0001
0
2.4004835237013053e-05


In [159]:
opacity = 0.8
colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

figs = {}
axs = {}
for application_name in application_names:
    fig, ax = plt.subplots(figsize=(10,9))
    figs[application_name] = fig
    axs[application_name] = ax
    fig.canvas.set_window_title(application_name)
    for i,security_type in enumerate(security_types):
        ax.plot(capacitor_sizes, base_charge_per_app[application_name]/(capacitor_sizes - get_security_charge(security_type, number_of_bytes_per_app[application_name])), label=security_type, color=colors[i], alpha=opacity)
#     ax.set_yscale('log')
#     ax.set_xscale('log')
    ax.set_xlabel('Capacitor Size [C]')
    ax.set_ylabel('Number of Checkpoints')
    ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
    fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
    ax.legend()
    fig.tight_layout()
    fig.show()
    break

## Scales

In [160]:
print("Cap Size:")
print(f"\tmin: {min(capacitor_sizes)}, max: {max(capacitor_sizes)}")

Cap Size:
	min: 0.001, max: 0.0019900000000000026


In [161]:
print("Base Charge:")
for application_name in application_names:
    print(f"\t{application_name}: {base_charge_per_app[application_name]}")

Base Charge:
	Kalman: 0.0001
	AR: 0.0002
	Dijkstra: 0.00030000000000000003
	Cuckoo: 0.0004
	Sorting: 0.0005


In [162]:
print("Security Charge:")
for security_type in security_types:
    print(f"\t{security_type} min: {get_security_charge(security_type, min(number_of_bytes_per_app.values()))},  max: {get_security_charge(security_type, max(number_of_bytes_per_app.values()))}")
# number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

Security Charge:
	None min: 0,  max: 0
	AES_Flash min: 6.771078408967512e-06,  max: 8.410964901737793e-05
	TruseZone_Flash min: 0,  max: 0


In [163]:
base_charge_per_app

{'Kalman': 0.0001,
 'AR': 0.0002,
 'Dijkstra': 0.00030000000000000003,
 'Cuckoo': 0.0004,
 'Sorting': 0.0005}

In [164]:
min(number_of_bytes_per_app.values())

12

In [138]:
get_security_charge("None", min(number_of_bytes_per_app.values()))

0

## Plot stuff

In [11]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Capacitor Size [C]")
ax.set_ylabel("Number of Checkpoints")

Text(0, 0.5, 'Number of Checkpoints')

In [12]:
capacitor_sizes = range(1,101)
def get_number_of_checkpoints(application_number):
    return [application_charges[application_number]/(capacitor_size - get_security_charge("trustzone_flash", application_number_of_bytes[application_number])) for capacitor_size in capacitor_sizes]

In [13]:
for application_number in range(5):
    ax.plot(capacitor_sizes, get_number_of_checkpoints(application_number), label=application_names[application_number])
ax.legend()
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.invert_xaxis()

In [14]:
# fig2 = plt.figure(figsize=(10, 6))
# ax2 = fig2.add_subplot(111, projection='3d')

# ax2.set_xlabel("Capacitor Size [C]")
# ax2.set_ylabel("Number of Bytes")
# ax2.set_zlabel("Number of Checkpoints")

In [15]:
# capacitor_sizes = range(1,1001)
# application_charge = 50
# number_of_bytes = range(1,1001)

In [16]:
# def get_number_of_checkpoints(application_charge, capacitor_size, number_of_bytes, security_type="trustzone_flash"):
#     return application_charge/(capacitor_size - get_security_charge(security_type, number_of_bytes))

In [17]:
# X, Y = np.meshgrid(capacitor_sizes, number_of_bytes)
# Z = np.zeros(shape=(1000,1000))
# for i,capacitor_size in enumerate(capacitor_sizes):
#     for j,number_of_byte in enumerate(number_of_bytes):
#         Z[i,j] = get_number_of_checkpoints(application_charge, capacitor_size, number_of_byte)
# #Z = np.array([np.array([get_number_of_checkpoints(capacitor_size,application_charge) for application_charge in application_charges]) for  capacitor_size in capacitor_sizes])
# Z.shape

In [18]:
# surf = ax2.plot_surface(X, Y, Z)